In [1]:
import os
import re
import string
import numpy as np
from unicodedata import normalize

In [23]:
#Função que vai criar uma nova lista de ids, mas agora os ids começarão do 0
#e serão com base nos títulos ordenados por ordem alfabética
def cria_lista_titulos(texto, lista_titulos):
    inicio = 0
    # continuamos para cada tag "<doc>" encontrada
    while texto.find('<doc id', inicio) != -1:
        inicio = texto.find('<doc id', inicio)
        
        # separamos o "id" do artigo
        ini_id = texto.find('id="', inicio)
        end_id = texto.find('"', ini_id + 4)
        doc_id = texto[ini_id + 4 : end_id]
        
        # separamos o "titulo" do artigo
        ini_ti = texto.find('title="', inicio)
        end_ti = texto.find('"', ini_ti + 7)
        titulo = texto[ini_ti+7:end_ti]
       
        lista_titulos.append((titulo, doc_id, 0))
        inicio = texto.find('</doc>', inicio)

files = os.listdir("english_wiki_corpus")

# Para cada arquivo no diretório avaliamos
# o texto e atualizamos a lista de titulos
l_titulos = []
k = 0
for file in files:
    f = open("english_wiki_corpus/" + file, errors="replace")
    texto = f.read()
    cria_lista_titulos(texto, l_titulos)
    k += 1
    print('Terminei a iteração ' + str(k))
#sorteamos os títulos em ordem alfabética
l_titulos.sort()
#criamos os novos valores dos ids
for tit in range(len(l_titulos)):
    l_titulos[tit] = (l_titulos[tit][0], l_titulos[tit][1], tit)
d_titulos = dict()
#criamos um dicionário onde cada título é uma chave com 2 valores, o id antigo e o id novo
for tit, ind1, ind2 in l_titulos:
    d_titulos[tit] = [ind1, ind2]

Terminei a iteração 1
Terminei a iteração 2
Terminei a iteração 3
Terminei a iteração 4
Terminei a iteração 5
Terminei a iteração 6
Terminei a iteração 7
Terminei a iteração 8
Terminei a iteração 9
Terminei a iteração 10
Terminei a iteração 11
Terminei a iteração 12
Terminei a iteração 13
Terminei a iteração 14
Terminei a iteração 15
Terminei a iteração 16
Terminei a iteração 17
Terminei a iteração 18
Terminei a iteração 19
Terminei a iteração 20
Terminei a iteração 21
Terminei a iteração 22
Terminei a iteração 23
Terminei a iteração 24
Terminei a iteração 25
Terminei a iteração 26
Terminei a iteração 27
Terminei a iteração 28
Terminei a iteração 29
Terminei a iteração 30
Terminei a iteração 31
Terminei a iteração 32
Terminei a iteração 33
Terminei a iteração 34
Terminei a iteração 35
Terminei a iteração 36
Terminei a iteração 37
Terminei a iteração 38
Terminei a iteração 39
Terminei a iteração 40
Terminei a iteração 41
Terminei a iteração 42
Terminei a iteração 43
Terminei a iteração 

In [28]:
## DEFINIMOS AS FUNÇÕES DE PROCESSAMENTO GERANDO
## O OBJETO DO INDEX DA LISTA INVERTIDA
index = {}

def update_index(text, id_text):
    # atualizamos o índice da lista invertida
    text = text.lower()
    text = re.sub('[^a-z0-9]+', ' ', text)
    for w in text.split():
        # limpeza das palavras: pode ser revisitado
        w = w.strip(string.punctuation)
        w = w.strip(",.;:\"+=")
        w = normalize('NFKD', w).encode('ASCII', 'ignore').decode('ASCII')
        
        # atualizamos o index com as palavras
        if w not in index.keys():
            index[w] = []
        index[w].append(id_text)

def analise_texto(texto):
    inicio = 0
    # continuamos para cada tag "<doc>" encontrada
    while texto.find('<doc id', inicio) != -1:
        inicio = texto.find('<doc id', inicio)
        
        # separamos o "titulo" do artigo
        ini_ti = texto.find('title="', inicio)
        end_ti = texto.find('"', ini_ti + 7)
        titulo = texto[ini_ti+7:end_ti]
        
        #pegamos o novo id usando o nosso dicionário de titulos e ids
        doc_id = d_titulos[titulo][1]
        
        # separamos o "corpo" do artigo
        ini_co = texto.find('">', inicio)
        fim_co = texto.find('</doc>', inicio)
        corpo = texto[ini_co + 2:fim_co]
            
        # chamamos a função de processamento
        update_index(titulo + " " + corpo, doc_id)
        
        inicio = fim_co

In [29]:
## ACESSAMOS OS DOCUMENTOS NA PASTA

# Lemos os arquivos do "english_texts"
files = os.listdir("english_wiki_corpus")
#files.remove('raw.en.tgz')

# Para cada arquivo no diretório avaliamos
# o texto e atualizamos o index de palavras
k = 0
for file in files:
    f = open("english_wiki_corpus/" + file, errors="replace")
    texto = f.read()
    analise_texto(texto)
    k += 1
    if k == 5:
        break
    print('Terminei a iteração ' + str(k))

Terminei a iteração 1
Terminei a iteração 2
Terminei a iteração 3
Terminei a iteração 4


In [30]:
## SALVAMOS UM ARQUIVO "index.txt" DA LISTA INVERTIDA
## FAZEMOS EM CADA LINHA: "palavra n id_1 id_2 ... id_n"

with open('index_teste.txt', 'a+') as file:
    for key in index.keys():
        if key.isalpha():
            s = str()
            index[key] = list(np.unique(index[key]))
            index[key].sort()
            for k in index[key]:
                s += ' ' + str(k)
            file.write(str(key) + " " + str(len(index[key])) + " " + s  + "\n")
    file.close()

In [9]:
def cria_docs(texto):
    inicio = 0
    # continuamos para cada tag "<doc>" encontrada
    while texto.find('<doc id', inicio) != -1:
        inicio = texto.find('<doc id', inicio)
        
        # separamos o "titulo" do artigo
        ini_ti = texto.find('title="', inicio)
        end_ti = texto.find('"', ini_ti + 7)
        titulo = texto[ini_ti+7:end_ti]
        
        # separamos o "corpo" do artigo
        ini_co = texto.find('">', inicio)
        fim_co = texto.find('</doc>', inicio)
        corpo = texto[ini_co + 2:fim_co]
        
        #pegamos o novo id
        doc_id = d_titulos[titulo][1]
        
        # geramos um arquivo "doc_id" cada artigo
        file = open("n_docs/doc_" + str(int(doc_id)//10000) + '/doc_' + str(doc_id), 'a+')
        file.write(titulo + "\n")
        file.write(corpo)
        file.close()
        
        inicio = fim_co

In [25]:
for k in range(136):
    os.mkdir('n_docs/doc_' + str(k))

In [26]:
## ACESSAMOS OS DOCUMENTOS NA PASTA

# Lemos os arquivos do "english_texts"
files = os.listdir("english_wiki_corpus")
#files.remove('raw.en.tgz')

# Para cada arquivo no diretório avaliamos
# o texto e atualizamos o index de palavras
k = 0
for file in files:
    f = open("english_wiki_corpus/" + file, errors="replace")
    texto = f.read()
    cria_docs(texto)
    k += 1
    print('Terminei a iteração ' + str(k))

Terminei a iteração 1
Terminei a iteração 2
Terminei a iteração 3
Terminei a iteração 4
Terminei a iteração 5
Terminei a iteração 6
Terminei a iteração 7
Terminei a iteração 8
Terminei a iteração 9
Terminei a iteração 10
Terminei a iteração 11
Terminei a iteração 12
Terminei a iteração 13
Terminei a iteração 14
Terminei a iteração 15
Terminei a iteração 16
Terminei a iteração 17
Terminei a iteração 18
Terminei a iteração 19
Terminei a iteração 20
Terminei a iteração 21
Terminei a iteração 22
Terminei a iteração 23
Terminei a iteração 24
Terminei a iteração 25
Terminei a iteração 26
Terminei a iteração 27
Terminei a iteração 28
Terminei a iteração 29
Terminei a iteração 30
Terminei a iteração 31
Terminei a iteração 32
Terminei a iteração 33
Terminei a iteração 34
Terminei a iteração 35
Terminei a iteração 36
Terminei a iteração 37
Terminei a iteração 38
Terminei a iteração 39
Terminei a iteração 40
Terminei a iteração 41
Terminei a iteração 42
Terminei a iteração 43
Terminei a iteração 

In [ ]:
## DEFINIMOS AS FUNÇÕES DE PROCESSAMENTO GERANDO
## O OBJETO DO INDEX DA LISTA INVERTIDA
index = []

def update_index(text):
    # atualizamos o índice da lista invertida
    text = text.lower()
    text = re.sub('[^a-z0-9]+', ' ', text)
    for w in text.split():
        # limpeza das palavras: pode ser revisitado
        w = w.strip(string.punctuation)
        w = w.strip(",.;:\"+=")
        w = normalize('NFKD', w).encode('ASCII', 'ignore').decode('ASCII')
        index.append(w)
    
def analise_texto(texto):
    inicio = 0
    # continuamos para cada tag "<doc>" encontrada
    while texto.find('<doc id', inicio) != -1:
        inicio = texto.find('<doc id', inicio)
        
        # separamos o "titulo" do ar
        
        # separamos o "corpo" do artigo
        ini_co = texto.find('">', inicio)
        fim_co = texto.find('</doc>', inicio)
        corpo = texto[ini_co + 2:fim_co]
            
        # chamamos a função de processamento
        update_index(titulo + " " + corpo, doc_id)
        
        inicio = fim_co